In [1]:
'''OUTPUT IS:
Connected with result code 0
All monitored macs over time are: 
['macs:0x6e68f6794f8d', 'macs:0x929ed5fc3184', 'macs:0xa28052f1dad9', 'macs:0xa3bcab39356:audioname', 'macs:0xbf1801ab4132', 'macs:0xe21150dd6238', 'macs:0xf209fa679649', 'macs:0xe21150dd6238:audioname', 'macs:0xa3bcab39356', 'macs:0x92bf3dbc3faa', 'macs:0xd6c8ac26ae1a', 'macs:0xa657c8c66eb6']
Received message '{"mac_address": "0xbf1801ab4132", "timestamp": 1677696079164, "prediction_confidence": 0.9941210150718689, "label": "NEU"}' on topic s295573
Timestamp is:  1677696079164
Label is:  NEU
Confidence is:  0.9941210150718689
'''

'OUTPUT IS:\nConnected with result code 0\nAll monitored macs over time are: \n[\'macs:0x6e68f6794f8d\', \'macs:0x929ed5fc3184\', \'macs:0xa28052f1dad9\', \'macs:0xa3bcab39356:audioname\', \'macs:0xbf1801ab4132\', \'macs:0xe21150dd6238\', \'macs:0xf209fa679649\', \'macs:0xe21150dd6238:audioname\', \'macs:0xa3bcab39356\', \'macs:0x92bf3dbc3faa\', \'macs:0xd6c8ac26ae1a\', \'macs:0xa657c8c66eb6\']\nReceived message \'{"mac_address": "0xbf1801ab4132", "timestamp": 1677696079164, "prediction_confidence": 0.9941210150718689, "label": "NEU"}\' on topic s295573\nTimestamp is:  1677696079164\nLabel is:  NEU\nConfidence is:  0.9941210150718689\n'

In [2]:
import paho.mqtt.client as mqtt
import uuid
import platform
import psutil
import time
import json
import redis
from redis.commands.json.path import Path

#PERSONAL PARAMETERS YOU CAN OBTAIN FROM YOUR REDIS ACCOUNT
REDIS_HOST = "****"
REDIS_PORT = 00000
REDIS_USERNAME = "****"
REDIS_PASSWORD = "****"

#ADD HERE THE TOPIC YOU WANT TO SUBSCRIBE TO
TOPIC = "****"



# Connect to Redis server
redis_client = redis.Redis(host=REDIS_HOST, port=REDIS_PORT, username=REDIS_USERNAME, password=REDIS_PASSWORD)
is_connected = redis_client.ping()
print(f'Redis is connected:  {is_connected}')

Redis is connected:  True


In [3]:
def safe_ts_create(key):
    try:
        # to create a redis time series
        redis_client.delete(key)
        redis_client.ts().create(key)
    except redis.ResponseError:
        pass

In [4]:
# Set up MQTT client and callbacks
def on_connect(client, userdata, flags, rc):
    print("Connected with result code "+ str(rc))
    client.subscribe(TOPIC)

def on_message(client, userdata, msg):
    #Extracting all the monitored macs
    keys = redis_client.keys('macs:*') #sort of regex 
    monitored_macs = []
    for key in keys:
        key = key.decode()
        monitored_macs.append(key)
        #print("Key is: ", key)
    
    message = msg.payload.decode() #returns a string with all the info needed
    #print(f"Received message '{message}' on topic {msg.topic}") #to check the message content
    

    # Extract all the data from message
    data = json.loads(message)
    timestamp = data['timestamp']
    label = data['label']
    confidence = data['prediction_confidence']
    mac_address = data['mac_address']
    #Creation of the timeseries name for each emotion for this specific mac address
    mac_Ang = mac_address + ':ANG'
    mac_Neu = mac_address + ':NEU'
    mac_Hap = mac_address + ':HAP'

    if f'macs:{mac_address}' not in monitored_macs:
        # New MAC address gets added if not present
        redis_client.json().set(f'macs:{mac_address}', Path.root_path(), mac_address) #adding to the server
        safe_ts_create(mac_Ang)
        safe_ts_create(mac_Neu)
        safe_ts_create(mac_Hap)
        print(f'Created new timeseries called:  {mac_Ang} - {mac_Neu} - {mac_Hap} . One for each emotion label')
        print(f'New MAC address: {mac_address} added to the list of addresses')

    #Prints for showing the content of the message
    print('Timestamp is: ', timestamp)
    print('Label is: ', label)
    print('Confidence is: ', confidence)
    
    #Adding the data to the correct timeseries
    name_of_timeseries = mac_address + ':' + label
    redis_client.ts().add(name_of_timeseries, timestamp, confidence)
    print(f'Added to the {name_of_timeseries} redis timeseries')    
    '''
    This solution could be used but we designed it in order to use TimeSeries so that you have one for each emotion label
    redis_client.json().set(f'macs:{mac_address}', Path.root_path(), message)

    redis.json().set(key, path, value)
    key is the name of the Redis key in which the JSON value will be stored.
    path is a path within the JSON document where the value will be stored.
    value is the JSON value to be stored at the specified path.
    '''   





client = mqtt.Client()
client.on_connect = on_connect
client.on_message = on_message

# Connect to MQTT broker 
client.connect('mqtt.eclipseprojects.io', 1883) #The MQTT broker can be changed

# Start MQTT event loop
client.loop_forever()

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=f648dc53-4e62-47c1-9023-194409755340' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>